In [27]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score

from finetune import Classifier
from finetune.config import get_default_config, get_small_model_config, GridSearchable, get_config

# Topic modelling only

### Set model hyperparameters through custom config
- Loads small model (6 layer transformer) config and then overwrites certain values
- Adding GridSearchable to a param makes model try every value of it in grid search
    - If you don't do grid search, model will use the default value provided

In [2]:
def get_custom_config():
    # conf = get_default_config()  
    conf = get_small_model_config()
    conf.l2_reg = GridSearchable(0.01, [0.0, 0.1, 0.01, 0.001]) 
    conf.lr = GridSearchable(6.25e-6, [6.25e-4, 6.25e-5, 6.25e-6])
    conf.n_epochs = GridSearchable(1, [1, 2, 3])
    conf.batch_size = 10
    conf.class_weights = 'linear'
    conf.oversample = False  # Do not use if using class_weights
    conf.low_memory_mode = True
    conf.val_size = 0.0  # Do not perform validation right now
    conf.val_interval = 10000  # Do not perform validation right now
    print("Config: ", conf)
    return conf

### Data loading
- Load from training data file which contains topic modelling and does not contain augmentation

In [3]:
# Load data
data_train = pd.read_csv("data_train_aug.csv")
print("Training data: {}".format(data_train.shape))
data_test = pd.read_csv("data_test_aug.csv")
print("Testing data: {}".format(data_test.shape))

trainX, trainY = data_train.Text, data_train.Targets
testX, testY = data_test.Text, data_test.Targets

Training data: (942, 2)
Testing data: (33, 2)


### Instantiate Classifier model
- Currently not performing grid search

In [8]:
model = Classifier(config=get_custom_config())

# Uncomment below to do grid search (And comment out above model)

# best_config = Classifier.finetune_grid_search_cv(trainX, trainY, n_splits=2, config=get_custom_config(), eval_fn=lambda pred, true: f1_score(pred, true), test_size=0.1)
# print("Best training parameters: \n{}".format(best_config))
# model = Classifier(config=best_config)

Config:  {'grid_searchable': {'n_epochs': [1, 2, 3], 'l2_reg': [0.0, 0.1, 0.01, 0.001], 'lr': [0.000625, 6.25e-05, 6.25e-06]}, 'batch_size': 10, 'visible_gpus': [0], 'n_epochs': 1, 'seed': 42, 'max_length': 512, 'weight_stddev': 0.02, 'chunk_long_sequences': False, 'low_memory_mode': True, 'interpolate_pos_embed': True, 'embed_p_drop': 0.1, 'attn_p_drop': 0.1, 'resid_p_drop': 0.1, 'clf_p_drop': 0.1, 'l2_reg': 0.01, 'vector_l2': False, 'regularize_deviation': False, 'b1': 0.9, 'b2': 0.999, 'epsilon': 1e-08, 'lr_schedule': 'warmup_linear', 'lr': 6.25e-06, 'lr_warmup': 0.002, 'max_grad_norm': 1, 'lm_loss_coef': 0.0, 'summarize_grads': False, 'verbose': True, 'val_size': 0.0, 'val_interval': 10000, 'val_window_size': 5, 'rolling_avg_decay': 0.99, 'lm_temp': 0.2, 'seq_num_heads': 16, 'pad_token': '<PAD>', 'subtoken_predictions': False, 'multi_label_sequences': False, 'multi_label_threshold': 0.5, 'autosave_path': None, 'tensorboard_folder': None, 'log_device_placement': False, 'soft_device_

### Train Classifier

In [9]:
model.finetune(np.array(trainX), np.array(trainY))

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


/home/ckjoshi9/miniconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### Evaluate predictions

In [10]:
# Compute metrics
predY = model.predict(testX)
accuracy = np.mean(predY == testY)
class_balance = np.mean(testY)
print('Test Accuracy: {:0.2f} for a {:0.2f} class balance'.format(accuracy, class_balance))
print("\nClassification Report:\n")
print(classification_report(model.label_encoder.transform(testY), model.label_encoder.transform(predY)))
print("\nConfusion Matrix:\n")
print(confusion_matrix(testY, predY))
print("\nF1-Score: ", f1_score(predY, testY))

Test Accuracy: 0.67 for a 0.36 class balance

Classification Report:

             precision    recall  f1-score   support

          0       0.66      1.00      0.79        21
          1       1.00      0.08      0.15        12

avg / total       0.78      0.67      0.56        33


Confusion Matrix:

[[21  0]
 [11  1]]

F1-Score:  0.15384615384615385


### Sample results (best run)

```
Training data: (107, 2)
Testing data: (33, 2)
Config:  {'grid_searchable': {'n_epochs': [1, 2, 3], 'l2_reg': [0.0, 0.1, 0.01, 0.001], 'lr': [0.000625, 6.25e-05, 6.25e-06]}, 'batch_size': 10, 'visible_gpus': [0], 'n_epochs': 1, 'seed': 42, 'max_length': 512, 'weight_stddev': 0.02, 'chunk_long_sequences': False, 'low_memory_mode': True, 'interpolate_pos_embed': True, 'embed_p_drop': 0.1, 'attn_p_drop': 0.1, 'resid_p_drop': 0.1, 'clf_p_drop': 0.1, 'l2_reg': 0.01, 'vector_l2': False, 'regularize_deviation': False, 'b1': 0.9, 'b2': 0.999, 'epsilon': 1e-08, 'lr_schedule': 'warmup_linear', 'lr': 6.25e-06, 'lr_warmup': 0.002, 'max_grad_norm': 1, 'lm_loss_coef': 0.0, 'summarize_grads': False, 'verbose': True, 'val_size': 0.0, 'val_interval': 10000, 'val_window_size': 5, 'rolling_avg_decay': 0.99, 'lm_temp': 0.2, 'seq_num_heads': 16, 'pad_token': '<PAD>', 'subtoken_predictions': False, 'multi_label_sequences': False, 'multi_label_threshold': 0.5, 'autosave_path': None, 'tensorboard_folder': None, 'log_device_placement': False, 'soft_device_placement': True, 'save_adam_vars': False, 'num_layers_trained': 6, 'train_embeddings': True, 'class_weights': 'linear', 'oversample': True, 'params_device': 'cpu', 'n_heads': 8, 'n_layer': 6, 'act_fn': 'gelu', 'n_embed': 512, 'base_model_path': '/home/ckjoshi9/depression_finetune/finetune/finetune/model/SmallBaseModel.jl'}
/home/ckjoshi9/depression_finetune/finetune/finetune/base.py:406: UserWarning: Model will only receive {} weight updates.  This may not be sufficient to find a good minima.Please consider lowering `config.batch_size` or providing more labeled training data to thet model.
  "Model will only receive {} weight updates.  This may not be sufficient to find a good minima."
/home/ckjoshi9/miniconda3/envs/finetune/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
                                                                                
Test Accuracy: 0.45 for a 0.36 class balance

Classification Report:

             precision    recall  f1-score   support

          0       0.71      0.24      0.36        21
          1       0.38      0.83      0.53        12

avg / total       0.59      0.45      0.42        33


Confusion Matrix:

[[ 5 16]
 [ 2 10]]

F1-Score:  0.5263157894736842
```

In [1]:
# model.save(<path>)
# del model

---

# Topic modelling + Data augmentation

In [2]:
def get_custom_config():
    # conf = get_default_config()  
    conf = get_small_model_config()
    conf.l2_reg = GridSearchable(0.01, [0.0, 0.1, 0.01, 0.001]) 
    conf.lr = GridSearchable(6.25e-6, [6.25e-4, 6.25e-5, 6.25e-6])
    conf.n_epochs = GridSearchable(1, [1, 2, 3])
    conf.batch_size = 2
    conf.class_weights ='linear'
    conf.oversample = False
    conf.low_memory_mode = True
    conf.val_size = 0.0
    conf.val_interval = 10000
    print("Config: ", conf)
    return conf

In [5]:
# Load data
data_train = pd.read_csv("data_train_aug.csv")   # IMPORTANT: USE CORRECT TRAINING DATA
print("Training data: {}".format(data_train.shape))
data_test = pd.read_csv("data_test_aug.csv")
print("Testing data: {}".format(data_test.shape))
trainX, trainY = data_train.Text, data_train.Targets
testX, testY = data_test.Text, data_test.Targets

Training data: (942, 2)
Testing data: (33, 2)


In [30]:
# Instantiate model
model = Classifier(get_config(**best_config))
# Train model
model.finetune(trainX, trainY)

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


/home/ckjoshi9/miniconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [33]:
# Compute metrics
predY = model.predict(trainX)
print("\nClassification Report:\n")
print(classification_report(model.label_encoder.transform(trainY), model.label_encoder.transform(predY)))
print("\nConfusion Matrix:\n")
print(confusion_matrix(trainY, predY))
print("\nF1-Score: ", f1_score(predY, trainY))


Classification Report:

             precision    recall  f1-score   support

          0       1.00      0.99      0.99       462
          1       0.99      1.00      0.99       480

avg / total       0.99      0.99      0.99       942


Confusion Matrix:

[[457   5]
 [  2 478]]

F1-Score:  0.9927310488058152


In [31]:
# Compute metrics
predY = model.predict(testX)
accuracy = np.mean(predY == testY)
class_balance = np.mean(testY)
print('Test Accuracy: {:0.2f} for a {:0.2f} class balance'.format(accuracy, class_balance))
print("\nClassification Report:\n")
print(classification_report(model.label_encoder.transform(testY), model.label_encoder.transform(predY)))
print("\nConfusion Matrix:\n")
print(confusion_matrix(testY, predY))
print("\nF1-Score: ", f1_score(predY, testY))

Test Accuracy: 0.67 for a 0.36 class balance

Classification Report:

             precision    recall  f1-score   support

          0       0.68      0.90      0.78        21
          1       0.60      0.25      0.35        12

avg / total       0.65      0.67      0.62        33


Confusion Matrix:

[[19  2]
 [ 9  3]]

F1-Score:  0.35294117647058826


### Sample Results (best run)

```
Training data: (534, 2)
Testing data: (33, 2)
Config:  {'grid_searchable': {'n_epochs': [1, 2, 3], 'l2_reg': [0.0, 0.1, 0.01, 0.001], 'lr': [0.000625, 6.25e-05, 6.25e-06]}, 'batch_size': 2, 'visible_gpus': [0], 'n_epochs': 1, 'seed': 42, 'max_length': 512, 'weight_stddev': 0.02, 'chunk_long_sequences': False, 'low_memory_mode': True, 'interpolate_pos_embed': True, 'embed_p_drop': 0.1, 'attn_p_drop': 0.1, 'resid_p_drop': 0.1, 'clf_p_drop': 0.1, 'l2_reg': 0.01, 'vector_l2': False, 'regularize_deviation': False, 'b1': 0.9, 'b2': 0.999, 'epsilon': 1e-08, 'lr_schedule': 'warmup_linear', 'lr': 6.25e-06, 'lr_warmup': 0.002, 'max_grad_norm': 1, 'lm_loss_coef': 0.0, 'summarize_grads': False, 'verbose': True, 'val_size': 0.0, 'val_interval': 10000, 'val_window_size': 5, 'rolling_avg_decay': 0.99, 'lm_temp': 0.2, 'seq_num_heads': 16, 'pad_token': '<PAD>', 'subtoken_predictions': False, 'multi_label_sequences': False, 'multi_label_threshold': 0.5, 'autosave_path': None, 'tensorboard_folder': None, 'log_device_placement': False, 'soft_device_placement': True, 'save_adam_vars': False, 'num_layers_trained': 6, 'train_embeddings': True, 'class_weights': 'linear', 'oversample': False, 'params_device': 'cpu', 'n_heads': 8, 'n_layer': 6, 'act_fn': 'gelu', 'n_embed': 512, 'base_model_path': '/home/ckjoshi9/depression_finetune/finetune/finetune/model/SmallBaseModel.jl'}
WARNING:tensorflow:From /home/ckjoshi9/miniconda3/envs/finetune/lib/python3.6/site-packages/tensorflow/python/framework/function.py:986: calling Graph.create_op (from tensorflow.python.framework.ops) with compute_shapes is deprecated and will be removed in a future version.
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
/home/ckjoshi9/miniconda3/envs/finetune/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
                                                                                
Test Accuracy: 0.85 for a 0.36 class balance

Classification Report:

             precision    recall  f1-score   support

          0       0.86      0.90      0.88        21
          1       0.82      0.75      0.78        12

avg / total       0.85      0.85      0.85        33


Confusion Matrix:

[[19  2]
 [ 3  9]]

F1-Score:  0.7826086956521738
```

In [ ]:
# model.save(<path>)
# del model

# Full Transcripts

In [ ]:
def get_custom_config():
    # conf = get_default_config()  
    conf = get_small_model_config()
    conf.l2_reg = GridSearchable(0.01, [0.0, 0.1, 0.01, 0.001]) 
    conf.lr = GridSearchable(6.25e-6, [6.25e-4, 6.25e-5, 6.25e-6])
    conf.n_epochs = GridSearchable(1, [1, 2, 3])
    conf.batch_size = 10
    conf.class_weights = 'linear'
    conf.oversample = False
    conf.low_memory_mode = True
    conf.val_size = 0.0
    conf.val_interval = 10000
    print("Config: ", conf)
    return conf

In [ ]:
# Load data
data_train = pd.read_csv("data_train_full.csv")  # IMPORTANT: USE CORRECT TRAINING DATA
print("Training data: {}".format(data_train.shape))
data_test = pd.read_csv("data_test_full.csv")
print("Testing data: {}".format(data_test.shape))
trainX, trainY = data_train.Text, data_train.Targets
testX, testY = data_test.Text, data_test.Targets

In [ ]:
# Instantiate model
model = Classifier(config=get_custom_config())

# Uncomment for grid search
# best_config = Classifier.finetune_grid_search_cv(trainX, trainY, n_splits=2, config=get_custom_config(), eval_fn=lambda pred, true: f1_score(pred, true), test_size=0.1)
# print("Best training parameters: \n{}".format(best_config))
# model = Classifier(config=best_config)

# Train model
model.finetune(trainX, trainY)

In [ ]:
# Compute metrics
predY = model.predict(testX)
accuracy = np.mean(predY == testY)
class_balance = np.mean(testY)
print('Test Accuracy: {:0.2f} for a {:0.2f} class balance'.format(accuracy, class_balance))
print("\nClassification Report:\n")
print(classification_report(model.label_encoder.transform(testY), model.label_encoder.transform(predY)))
print("\nConfusion Matrix:\n")
print(confusion_matrix(testY, predY))
print("\nF1-Score: ", f1_score(predY, testY))

### Sample Results (best run)

```
/home/ckjoshi9/depression_finetune/finetune/finetune/encoding.py:307: UserWarning: Some examples are longer than the max_length. Please trim documents or increase `max_length`. Fallback behaviour is to use the first 510 byte-pair encoded tokens
  "Fallback behaviour is to use the first {} byte-pair encoded tokens".format(max_length - 2)
Training data: (107, 2)
Testing data: (35, 2)
Config:  {'grid_searchable': {'n_epochs': [1, 2, 3], 'l2_reg': [0.0, 0.1, 0.01, 0.001], 'lr': [0.000625, 6.25e-05, 6.25e-06]}, 'batch_size': 10, 'visible_gpus': [0], 'n_epochs': 1, 'seed': 42, 'max_length': 512, 'weight_stddev': 0.02, 'chunk_long_sequences': False, 'low_memory_mode': True, 'interpolate_pos_embed': True, 'embed_p_drop': 0.1, 'attn_p_drop': 0.1, 'resid_p_drop': 0.1, 'clf_p_drop': 0.1, 'l2_reg': 0.01, 'vector_l2': False, 'regularize_deviation': False, 'b1': 0.9, 'b2': 0.999, 'epsilon': 1e-08, 'lr_schedule': 'warmup_linear', 'lr': 6.25e-06, 'lr_warmup': 0.002, 'max_grad_norm': 1, 'lm_loss_coef': 0.0, 'summarize_grads': False, 'verbose': True, 'val_size': 0.0, 'val_interval': 10000, 'val_window_size': 5, 'rolling_avg_decay': 0.99, 'lm_temp': 0.2, 'seq_num_heads': 16, 'pad_token': '<PAD>', 'subtoken_predictions': False, 'multi_label_sequences': False, 'multi_label_threshold': 0.5, 'autosave_path': None, 'tensorboard_folder': None, 'log_device_placement': False, 'soft_device_placement': True, 'save_adam_vars': False, 'num_layers_trained': 6, 'train_embeddings': True, 'class_weights': 'linear', 'oversample': False, 'params_device': 'cpu', 'n_heads': 8, 'n_layer': 6, 'act_fn': 'gelu', 'n_embed': 512, 'base_model_path': '/home/ckjoshi9/depression_finetune/finetune/finetune/model/SmallBaseModel.jl'}
/home/ckjoshi9/depression_finetune/finetune/finetune/encoding.py:246: UserWarning: Document is longer than max length allowed, trimming document to 512 tokens.
  max_length
/home/ckjoshi9/depression_finetune/finetune/finetune/base.py:406: UserWarning: Model will only receive {} weight updates.  This may not be sufficient to find a good minima.Please consider lowering `config.batch_size` or providing more labeled training data to thet model.
  "Model will only receive {} weight updates.  This may not be sufficient to find a good minima."
/home/ckjoshi9/miniconda3/envs/finetune/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
                      
Test Accuracy: 0.51 for a 0.34 class balance

Classification Report:

             precision    recall  f1-score   support

          0       0.69      0.48      0.56        23
          1       0.37      0.58      0.45        12

avg / total       0.58      0.51      0.53        35


Confusion Matrix:

[[11 12]
 [ 5  7]]

F1-Score:  0.4516129032258065
```

In [2]:
# model.save(<path>)
# del model